In [1]:
import pandas as pd
import pickle

In [2]:
with open('E:\Aegis\NLP\Project/yelp_data_subset.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
dtest = data

In [4]:
#Converting list to array
import numpy as np
myarray = np.asarray(dtest)

In [5]:
#myarray

In [6]:
#Converting list to data frame using Pandas

df = pd.DataFrame(dtest, columns=['votes', 'user_id', 'review_id', 'text','business_id','date','type'])

In [7]:
df.head()

,votes,user_id,review_id,text,business_id,date,type
0,"{u'funny': 0, u'useful': 0, u'cool': 0}",PUFPaY9KxDAcGqfsorJp3Q,Ya85v4eqdd6k9Od8HbQjyA,"Mr Hoagie is an institution. Walking in, it do...",5UmKMjUEUNdYWqANhGckJw,2012-08-01,review
1,"{u'funny': 0, u'useful': 0, u'cool': 0}",Iu6AxdBYGR4A0wspR9BYHA,KPvLNJ21_4wbYNctrOwWdQ,Excellent food. Superb customer service. I mis...,5UmKMjUEUNdYWqANhGckJw,2014-02-13,review
2,"{u'funny': 1, u'useful': 1, u'cool': 0}",auESFwWvW42h6alXgFxAXQ,fFSoGV46Yxuwbr3fHNuZig,Yes this place is a little out dated and not o...,5UmKMjUEUNdYWqANhGckJw,2015-10-31,review
3,"{u'funny': 0, u'useful': 0, u'cool': 0}",qiczib2fO_1VBG8IoCGvVg,pVMIt0a_QsKtuDfWVfSk2A,PROS: Italian hoagie was delicious. Friendly ...,5UmKMjUEUNdYWqANhGckJw,2015-12-26,review
4,"{u'funny': 0, u'useful': 1, u'cool': 0}",qEE5EvV-f-s7yHC0Z4ydJQ,AEyiQ_Y44isJmNbMTyoMKQ,First the only reason this place could possibl...,5UmKMjUEUNdYWqANhGckJw,2016-04-08,review


In [8]:
df.shape

(5503, 7)

In [9]:
#Data extraction is completed.
#
#Check for missing values in the data
#
df.apply(lambda x: sum(x.isnull()))

votes          0
user_id        0
review_id      0
text           0
business_id    0
date           0
type           0
dtype: int64

In [10]:
#There are no missing values in the data

In [11]:
df_ = df

In [12]:
del df_['votes']
#Look at no.of categorical variables in the data
df_.apply(lambda x: len(x.unique()))

user_id        3519
review_id      5503
text           5499
business_id     311
date           2185
type              1
dtype: int64

In [13]:
#____Classification of reviews_____#
#Represent the text in numerical attributes


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
vectorizer = TfidfVectorizer(max_df = 0.5, min_df= 2, stop_words='english')


In [15]:
Reviews_text = df_['text']

In [16]:
X = vectorizer.fit_transform(Reviews_text)
X

<5503x10619 sparse matrix of type '<type 'numpy.float64'>'
	with 250590 stored elements in Compressed Sparse Row format>

In [17]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters= 5, init='k-means++', max_iter = 100, n_init=1, verbose=True)

In [18]:
km.fit(X)

Initialization complete
Iteration  0, inertia 10265.515
Iteration  1, inertia 5331.160
Iteration  2, inertia 5311.718
Iteration  3, inertia 5302.190
Iteration  4, inertia 5299.678
Iteration  5, inertia 5298.100
Iteration  6, inertia 5296.804
Iteration  7, inertia 5295.688
Iteration  8, inertia 5294.750
Iteration  9, inertia 5293.726
Iteration 10, inertia 5292.511
Iteration 11, inertia 5291.073
Iteration 12, inertia 5289.129
Iteration 13, inertia 5287.135
Iteration 14, inertia 5286.612
Iteration 15, inertia 5286.403
Iteration 16, inertia 5286.168
Iteration 17, inertia 5285.964
Iteration 18, inertia 5285.908
Iteration 19, inertia 5285.887
Iteration 20, inertia 5285.883
Iteration 21, inertia 5285.880
Converged at iteration 21


KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=5, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=True)

In [19]:
import numpy as np
np.unique(km.labels_, return_counts=True)

(array([0, 1, 2, 3, 4]), array([ 345, 2629,  283,  872, 1374], dtype=int64))

In [20]:
#Remove the same words present in among all the cluster and See the top 10 words
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk

In [21]:
_stopwords = set(stopwords.words('english') + list(punctuation) + ["'s","''"])

In [22]:
text ={}
for i, cluster in enumerate(km.labels_):
    oneDocument = Reviews_text[i]
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else:
        text[cluster] += oneDocument

In [23]:
keywords = {}
counts ={}
for cluster in range(5):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent = [word for word in word_sent if word not in _stopwords]
    freq= FreqDist(word_sent)
    keywords[cluster] =nlargest(100, freq, key = freq.get)
    counts[cluster] = freq

In [24]:
unique_keys ={}
for cluster in range(5):
    other_clusters = list(set(range(5)) - set([cluster]))
    keys_other_clusters = set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))
    unique= set(keywords[cluster]) -keys_other_clusters
    unique_keys[cluster] = nlargest(100, unique, key=counts[cluster].get)

In [25]:
keywords[0]

[u'pizza',
 u'cheese',
 u"n't",
 u'good',
 u'place',
 u'crust',
 u'like',
 u'sauce',
 u'get',
 u'one',
 u'would',
 u'order',
 u'...',
 u"'ve",
 u'great',
 u'cold',
 u'really',
 u'time',
 u'toppings',
 u'also',
 u'best',
 u'go',
 u'love',
 u'beto',
 u'pittsburgh',
 u'ordered',
 u'back',
 u'food',
 u'eat',
 u'``',
 u'delivery',
 u"'m",
 u'slice',
 u'try',
 u'even',
 u'much',
 u'little',
 u'got',
 u'pretty',
 u'always',
 u'ever',
 u'first',
 u'pepperoni',
 u'better',
 u'take',
 u'well',
 u'since',
 u'slices',
 u'people',
 u'never',
 u'could',
 u'delicious',
 u'nice',
 u'favorite',
 u'hoagie',
 u'know',
 u'hot',
 u'top',
 u'make',
 u'way',
 u'two',
 u'night',
 u'years',
 u'want',
 u'extra',
 u'service',
 u'chicken',
 u'ca',
 u'say',
 u'right',
 u'around',
 u'tried',
 u'fresh',
 u'still',
 u'thing',
 u'bit',
 u'home',
 u'minutes',
 u'definitely',
 u'area',
 u'different',
 u'us',
 u"'re",
 u'melted',
 u'wait',
 u'pizzas',
 u'small',
 u'nothing',
 u'salad',
 u'worth',
 u'bad',
 u"'d",
 u'eati

In [26]:
df_.head()

,user_id,review_id,text,business_id,date,type
0,PUFPaY9KxDAcGqfsorJp3Q,Ya85v4eqdd6k9Od8HbQjyA,"Mr Hoagie is an institution. Walking in, it do...",5UmKMjUEUNdYWqANhGckJw,2012-08-01,review
1,Iu6AxdBYGR4A0wspR9BYHA,KPvLNJ21_4wbYNctrOwWdQ,Excellent food. Superb customer service. I mis...,5UmKMjUEUNdYWqANhGckJw,2014-02-13,review
2,auESFwWvW42h6alXgFxAXQ,fFSoGV46Yxuwbr3fHNuZig,Yes this place is a little out dated and not o...,5UmKMjUEUNdYWqANhGckJw,2015-10-31,review
3,qiczib2fO_1VBG8IoCGvVg,pVMIt0a_QsKtuDfWVfSk2A,PROS: Italian hoagie was delicious. Friendly ...,5UmKMjUEUNdYWqANhGckJw,2015-12-26,review
4,qEE5EvV-f-s7yHC0Z4ydJQ,AEyiQ_Y44isJmNbMTyoMKQ,First the only reason this place could possibl...,5UmKMjUEUNdYWqANhGckJw,2016-04-08,review


In [27]:
df_['Cluster0'] = 0
df_['Cluster1'] = 0
df_['Cluster2'] = 0
df_['Cluster3'] = 0
df_['Cluster4'] = 0
#
df_['Clust0_Sent'] = "NEUTRAL"
df_['Clust1_Sent'] = "NEUTRAL"
df_['Clust2_Sent'] = "NEUTRAL"
df_['Clust3_Sent'] = "NEUTRAL"
df_['Clust4_Sent'] = "NEUTRAL"

In [28]:
df_.head(2)

,user_id,review_id,text,business_id,date,type,Cluster0,Cluster1,Cluster2,Cluster3,Cluster4,Clust0_Sent,Clust1_Sent,Clust2_Sent,Clust3_Sent,Clust4_Sent
0,PUFPaY9KxDAcGqfsorJp3Q,Ya85v4eqdd6k9Od8HbQjyA,"Mr Hoagie is an institution. Walking in, it do...",5UmKMjUEUNdYWqANhGckJw,2012-08-01,review,0,0,0,0,0,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
1,Iu6AxdBYGR4A0wspR9BYHA,KPvLNJ21_4wbYNctrOwWdQ,Excellent food. Superb customer service. I mis...,5UmKMjUEUNdYWqANhGckJw,2014-02-13,review,0,0,0,0,0,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL


In [29]:
length_df = len(df_['text'])
length_df

5503

In [30]:
print len(sent_tokenize(df_.text[0]))
print sent_tokenize(df_.text[0])[1]

5
Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food.


In [31]:
df_.head()

,user_id,review_id,text,business_id,date,type,Cluster0,Cluster1,Cluster2,Cluster3,Cluster4,Clust0_Sent,Clust1_Sent,Clust2_Sent,Clust3_Sent,Clust4_Sent
0,PUFPaY9KxDAcGqfsorJp3Q,Ya85v4eqdd6k9Od8HbQjyA,"Mr Hoagie is an institution. Walking in, it do...",5UmKMjUEUNdYWqANhGckJw,2012-08-01,review,0,0,0,0,0,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
1,Iu6AxdBYGR4A0wspR9BYHA,KPvLNJ21_4wbYNctrOwWdQ,Excellent food. Superb customer service. I mis...,5UmKMjUEUNdYWqANhGckJw,2014-02-13,review,0,0,0,0,0,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
2,auESFwWvW42h6alXgFxAXQ,fFSoGV46Yxuwbr3fHNuZig,Yes this place is a little out dated and not o...,5UmKMjUEUNdYWqANhGckJw,2015-10-31,review,0,0,0,0,0,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
3,qiczib2fO_1VBG8IoCGvVg,pVMIt0a_QsKtuDfWVfSk2A,PROS: Italian hoagie was delicious. Friendly ...,5UmKMjUEUNdYWqANhGckJw,2015-12-26,review,0,0,0,0,0,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
4,qEE5EvV-f-s7yHC0Z4ydJQ,AEyiQ_Y44isJmNbMTyoMKQ,First the only reason this place could possibl...,5UmKMjUEUNdYWqANhGckJw,2016-04-08,review,0,0,0,0,0,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL


In [32]:
#df_.head(3)

In [33]:
df_.to_csv("yelp_int2.csv", encoding='utf-8')

In [ ]:
df_.shape

In [58]:
#unique_keys

#Amazon Food reviews

In [34]:
Amazon = pd.read_csv(r"E:\Aegis\NLP\Amazon_Food_review\amazon-fine-foods/Reviews.csv")
Amazon.shape

(568454, 10)

In [35]:
Amazon_ = Amazon

In [36]:
Amazon_.apply(lambda x: sum(x.isnull()))

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   26
Text                       0
dtype: int64

In [37]:
del Amazon_['ProfileName']
del Amazon_['HelpfulnessNumerator']
del Amazon_['HelpfulnessDenominator']
del Amazon_['Time']
del Amazon_['Summary']

In [38]:
Amazon_.head()

,Id,ProductId,UserId,Score,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,5,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,1,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,4,This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,2,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,5,Great taffy at a great price. There was a wid...


In [39]:
Amazon_['Sentiment'] ="A"

In [40]:
Amazon_['Sentiment'] [Amazon_['Score'] >= 4 ] ="positive"
Amazon_['Sentiment'] [Amazon_['Score'] == 3 ] ="neutral"
Amazon_['Sentiment'] [Amazon_['Score'] <= 2 ] ="negative"

C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [41]:
Amazon_.head()

,Id,ProductId,UserId,Score,Text,Sentiment
0,1,B001E4KFG0,A3SGXH7AUHU8GW,5,I have bought several of the Vitality canned d...,positive
1,2,B00813GRG4,A1D87F6ZCVE5NK,1,Product arrived labeled as Jumbo Salted Peanut...,negative
2,3,B000LQOCH0,ABXLMWJIXXAIN,4,This is a confection that has been around a fe...,positive
3,4,B000UA0QIQ,A395BORC6FGVXV,2,If you are looking for the secret ingredient i...,negative
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,5,Great taffy at a great price. There was a wid...,positive


In [42]:
Amazon_['Sentiment'][Amazon_['Score'] == 5].count()

363122

In [43]:
print Amazon_['Sentiment'][Amazon_['Sentiment'] == "negative"].count()
print Amazon_['Sentiment'][Amazon_['Sentiment'] == "neutral"].count()

82037
42640


In [44]:
fixed_length =10000

In [45]:
import numpy as np

In [46]:
positive_indices = np.array(Amazon_[Amazon_.Sentiment=="positive"].index)
negative_indices = np.array(Amazon_[Amazon_.Sentiment=="negative"].index)
#normal_indices = Amazon_[Amazon_['Sentiment']=="negative"].index
#now pick random samples
random_pos_indices = np.random.choice(positive_indices, fixed_length, replace=False)
random_neg_indices = np.random.choice(negative_indices, fixed_length, replace=False)
random_pos_indices = np.array(random_pos_indices)
random_neg_indices = np.array(random_neg_indices)

#Now Combine both indices
appended_indices = np.concatenate([random_pos_indices, random_neg_indices])

#
appended_dataset = Amazon_.iloc[appended_indices,:]
appended_dataset.head()

,Id,ProductId,UserId,Score,Text,Sentiment
299008,299009,B000HPZM74,A2IMF02ESIYFCK,5,its they're treat after getting the bark colla...,positive
201564,201565,B007R900WA,A324GNNAEX3ZRZ,5,This holder is great. If your cabinets are ta...,positive
550569,550570,B001P7539A,A1A4W9XNAHMSGS,5,"This popcorn is the best popcorn ever, it is j...",positive
278081,278082,B003O7T87C,ARM093ZVUMP9U,5,I have bought this product regularly for about...,positive
33187,33188,B000SATIZA,A997PUM414JUE,5,I use the hibiscus flower tea thus:<br /><br /...,positive


In [47]:
pd.value_counts(appended_dataset.Sentiment, sort=True).sort_index()

negative    10000
positive    10000
Name: Sentiment, dtype: int64

In [48]:
from bs4 import BeautifulSoup  
import nltk
import re
from nltk.corpus import stopwords

In [49]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words )) 

In [50]:
print appended_dataset.shape

(20000, 6)


In [51]:
print appended_dataset["Text"].size
appended_dataset.iloc[4]['Text']

20000


'I use the hibiscus flower tea thus:<br /><br />In a slightly larger than two quart (half gallon) container place 3/4 to 1 cup hibiscus flowers and 3/4-1 tablespoon dried stevia leaves.<br /><br />Add two quarts of boiling water.<br /><br />Let sit for at least 10 minutes.  Drink some hot (if you like).  When it is cool, strain through a fine sieve or mesh into a refrigerator container and enjoy cold.<br /><br />My daughters (12 & 14) enjoy drinking this cold over ice instead of soda and take it to school in the morning in their water bottles.  I drink it cold after running, working in the yard, or going to the gym as it is extraordinarily refreshing cold.  It also tends to keep me from snacking between meals.  We go through about 2-3 gallons/week, mostly cold.<br /><br />Other recommendations:  It stains, so pour it over the sink or something that won\'t stain.  It is very convenient online, but can also be purchased from many local Mexican markets from the same section as dried peppe

In [52]:
num_reviews = appended_dataset["Text"].size
clean_train_reviews = []

for i in xrange( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( appended_dataset.iloc[i]["Text"] ) )

C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\Users\Ajith Vajrala\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [53]:
from sklearn.feature_extraction.text import CountVectorizer
# 
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 
#
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
#train_data_features = train_data_features.toarray()

In [54]:
vocab = vectorizer.get_feature_names()
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, appended_dataset["Sentiment"] )

Training the random forest...


In [55]:
clean_test_reviews = [] 
clean_review = review_to_words(     sent_tokenize(df_['text'][1])[0]     )
clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)
print result

['positive']


In [56]:
length_df

5503

In [66]:
print df_['Clust0_Sent'][0:10,]
print df_['Clust1_Sent'][0:10,]
df_.head(10)

0    [positive]
1    [positive]
2    [negative]
3    [negative]
4    [negative]
5    [negative]
6    [negative]
7    [negative]
8    [positive]
9    [positive]
Name: Clust0_Sent, dtype: object
0    [positive]
1       NEUTRAL
2    [negative]
3       NEUTRAL
4       NEUTRAL
5    [negative]
6    [negative]
7    [negative]
8    [positive]
9    [positive]
Name: Clust1_Sent, dtype: object


,user_id,review_id,text,business_id,date,type,Cluster0,Cluster1,Cluster2,Cluster3,Cluster4,Clust0_Sent,Clust1_Sent,Clust2_Sent,Clust3_Sent,Clust4_Sent
0,PUFPaY9KxDAcGqfsorJp3Q,Ya85v4eqdd6k9Od8HbQjyA,"Mr Hoagie is an institution. Walking in, it do...",5UmKMjUEUNdYWqANhGckJw,2012-08-01,review,1,1,0,0,0,[positive],[positive],NEUTRAL,NEUTRAL,NEUTRAL
1,Iu6AxdBYGR4A0wspR9BYHA,KPvLNJ21_4wbYNctrOwWdQ,Excellent food. Superb customer service. I mis...,5UmKMjUEUNdYWqANhGckJw,2014-02-13,review,1,0,0,0,0,[positive],NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
2,auESFwWvW42h6alXgFxAXQ,fFSoGV46Yxuwbr3fHNuZig,Yes this place is a little out dated and not o...,5UmKMjUEUNdYWqANhGckJw,2015-10-31,review,1,1,0,0,0,[negative],[negative],NEUTRAL,NEUTRAL,NEUTRAL
3,qiczib2fO_1VBG8IoCGvVg,pVMIt0a_QsKtuDfWVfSk2A,PROS: Italian hoagie was delicious. Friendly ...,5UmKMjUEUNdYWqANhGckJw,2015-12-26,review,1,0,0,0,0,[negative],NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
4,qEE5EvV-f-s7yHC0Z4ydJQ,AEyiQ_Y44isJmNbMTyoMKQ,First the only reason this place could possibl...,5UmKMjUEUNdYWqANhGckJw,2016-04-08,review,1,0,0,0,0,[negative],NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
5,jBoH6qKGO7wdYyg_YjBcQA,V-bqYx62zpxfH2oFkzXPzw,"Normally, I do not do reviews of an establishm...",5UmKMjUEUNdYWqANhGckJw,2016-04-10,review,1,1,0,0,0,[negative],[negative],NEUTRAL,NEUTRAL,NEUTRAL
6,bWrodc7hN_T4q2r-rolrOA,3saY_LVFUpkAfd9t86VNdw,I like this place a lot. It's a good toasted h...,5UmKMjUEUNdYWqANhGckJw,2016-05-11,review,1,1,0,0,0,[negative],[negative],NEUTRAL,NEUTRAL,NEUTRAL
7,uK8tzraOp4M5u3uYrqIBXg,Di3exaUCFNw1V4kSNW5pgA,All the food is great here. But the best thing...,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,review,1,1,0,0,0,[negative],[negative],NEUTRAL,NEUTRAL,NEUTRAL
8,I_47G-R2_egp7ME5u_ltew,0Lua2-PbqEQMjD9r89-asw,We checked this place out this past Monday for...,UsFtqoBl7naz8AVUBZMjQQ,2014-03-29,review,1,1,0,0,0,[positive],[positive],NEUTRAL,NEUTRAL,NEUTRAL
9,PP_xoMSYlGr2pb67BbqBdA,7N9j5YbBHBW6qguE5DAeyA,Wing sauce is like water. Pretty much a lot of...,UsFtqoBl7naz8AVUBZMjQQ,2014-10-29,review,1,1,0,0,0,[positive],[positive],NEUTRAL,NEUTRAL,NEUTRAL


In [58]:
#Cluster0

for i in range(10):
    new_sent =[]
    for j in  range(len(sent_tokenize(df_['text'][i]))):
        #
        new_sent = sent_tokenize(df_.text[i])[j]
        new_words = [x.lower() for x in word_tokenize(new_sent)]
        result = []

        for k in range(len(unique_keys[0])):
            if unique_keys[0][k] in new_words:
                df_['Cluster0'][i] = 1
                
            
        
    if df_['Cluster0'][i] == 1:
        clean_test_reviews = [] 
        clean_review = review_to_words(new_sent)
        clean_test_reviews.append( clean_review )
        # Get a bag of words for the test set, and convert to a numpy array
        test_data_features = vectorizer.transform(clean_test_reviews)
        test_data_features = test_data_features.toarray()
        # Use the random forest to make sentiment label predictions
        result = forest.predict(test_data_features)
        print result
        df_['Clust0_Sent'][i] = result
            

C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['positive']
['positive']
['negative']
['negative']
['negative']
['negative']
['negative']
['negative']
['positive']
['positive']


In [59]:
#Cluster1

for i in range(10):
    new_sent =[]
    for j in  range(len(sent_tokenize(df_['text'][i]))):
        #
        new_sent = sent_tokenize(df_.text[i])[j]
        new_words = [x.lower() for x in word_tokenize(new_sent)]
        
        for k in range(len(unique_keys[1])):
            if unique_keys[1][k] in new_words:
                df_['Cluster1'][i] = 1
                break
                #break
             
        
    clean_test_reviews = []
    result = []
    if df_['Cluster1'][i] == 1:
            
        clean_test_reviews = [] 
        clean_review = review_to_words(new_sent)
        clean_test_reviews.append( clean_review )

        # Get a bag of words for the test set, and convert to a numpy array
        test_data_features = vectorizer.transform(clean_test_reviews)
        test_data_features = test_data_features.toarray()
        # Use the random forest to make sentiment label predictions
        result = forest.predict(test_data_features)
        print result
        df_['Clust1_Sent'][i] = result

C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['positive']
['negative']
['negative']
['negative']
['negative']
['positive']
['positive']


In [87]:
for i in range(length_df):
    new_sent =[]
    for j in  range(len(sent_tokenize(df_['text'][i]))):
        #
        new_sent = sent_tokenize(df_.text[i])[j]
        new_words = [x.lower() for x in word_tokenize(new_sent)]
        
        for k in range(len(unique_keys[2])):
            if unique_keys[2][k] in new_words:
                df_['Cluster2'][i] = 1
                break
                break
        clean_test_reviews = []     
        result = []
    
    if df_['Cluster2'][i] == 1:
            
        clean_test_reviews = [] 
        clean_review = review_to_words(new_sent)
        clean_test_reviews.append( clean_review )

        # Get a bag of words for the test set, and convert to a numpy array
        test_data_features = vectorizer.transform(clean_test_reviews)
        test_data_features = test_data_features.toarray()
        # Use the random forest to make sentiment label predictions
        result = forest.predict(test_data_features)
        #print result
        df_['Clust2_Sent'][i] = result

C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://eatinginpittsburgh.com/2010/07/07/ive-got-an-itch-for-kitsch-quiet-storm-in-bloomfield/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_m

In [89]:
for i in range(length_df):
    new_sent =[]
    for j in  range(len(sent_tokenize(df_['text'][i]))):
        #
        new_sent = sent_tokenize(df_.text[i])[j]
        new_words = [x.lower() for x in word_tokenize(new_sent)]
        
        for k in range(len(unique_keys[3])):
            if unique_keys[3][k] in new_words:
                df_['Cluster3'][i] = 1
                break
                break
    clean_test_reviews = []     
    result = []
    
    if df_['Cluster3'][i] == 1:
            
        clean_test_reviews = [] 
        clean_review = review_to_words(new_sent)
        clean_test_reviews.append( clean_review )

        # Get a bag of words for the test set, and convert to a numpy array
        test_data_features = vectorizer.transform(clean_test_reviews)
        test_data_features = test_data_features.toarray()
        # Use the random forest to make sentiment label predictions
        result = forest.predict(test_data_features)
        #print result
        df_['Clust3_Sent'][i] = result

C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [90]:
for i in range(length_df):
    new_sent =[]
    for j in  range(len(sent_tokenize(df_['text'][i]))):
        #
        new_sent = sent_tokenize(df_.text[i])[j]
        new_words = [x.lower() for x in word_tokenize(new_sent)]
        
        for k in range(len(unique_keys[4])):
            if unique_keys[4][k] in new_words:
                df_['Cluster4'][i] = 1
                break
                break
    clean_test_reviews = []     
    result = []
    
    if df_['Cluster4'][i] == 1:
            
        clean_test_reviews = [] 
        clean_review = review_to_words(new_sent)
        clean_test_reviews.append( clean_review )

        # Get a bag of words for the test set, and convert to a numpy array
        test_data_features = vectorizer.transform(clean_test_reviews)
        test_data_features = test_data_features.toarray()
        # Use the random forest to make sentiment label predictions
        result = forest.predict(test_data_features)
        #print result
        df_['Clust4_Sent'][i] = result

C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ajith Vajrala\Anaconda2\lib\site-packages\ipykernel\__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [91]:
df_.to_csv("yelp_final.csv", encoding='utf-8')

In [92]:
df_.head(2)

,user_id,review_id,text,business_id,date,type,Cluster0,Cluster1,Cluster2,Cluster3,Cluster4,Clust0_Sent,Clust1_Sent,Clust2_Sent,Clust3_Sent,Clust4_Sent
0,PUFPaY9KxDAcGqfsorJp3Q,Ya85v4eqdd6k9Od8HbQjyA,"Mr Hoagie is an institution. Walking in, it do...",5UmKMjUEUNdYWqANhGckJw,2012-08-01,review,1,1,1,0,0,[positive],[positive],[positive],NEUTRAL,NEUTRAL
1,Iu6AxdBYGR4A0wspR9BYHA,KPvLNJ21_4wbYNctrOwWdQ,Excellent food. Superb customer service. I mis...,5UmKMjUEUNdYWqANhGckJw,2014-02-13,review,0,1,0,0,1,NEUTRAL,[positive],NEUTRAL,NEUTRAL,[positive]


In [94]:
df_.tail()

,user_id,review_id,text,business_id,date,type,Cluster0,Cluster1,Cluster2,Cluster3,Cluster4,Clust0_Sent,Clust1_Sent,Clust2_Sent,Clust3_Sent,Clust4_Sent
5498,n5xDCeUAqylS6ShVEfQj1Q,RFZneP0UpWUi97pxiwZ0mA,"I had the Italian jombot here it was so good, ...",2Igiyhi82R8_QaaPplRmCw,2014-09-10,review,0,0,0,0,0,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
5499,Xk8ZXjwQf52U_P8RX5o9lQ,ED0aPWj68SiXaTC0genCmg,Came here on a Sunday morning. There was alrea...,2Igiyhi82R8_QaaPplRmCw,2014-09-14,review,1,1,0,1,1,[positive],[positive],NEUTRAL,[positive],[positive]
5500,tcB-GnnnBbpLnJ2D_4BYVg,qZAO0sqD6rG3HENy5LTnCw,I couldn't choose between pumpkin or banana pa...,2Igiyhi82R8_QaaPplRmCw,2014-09-15,review,1,1,0,1,0,[positive],[positive],NEUTRAL,[positive],NEUTRAL
5501,8v67cG2_5R86vn4Grx1sPA,W81w8XY0pC7Gt41iP6S-Dg,So with all the hype I was expecting fireworks...,2Igiyhi82R8_QaaPplRmCw,2014-09-15,review,1,1,1,1,0,[negative],[negative],[negative],[negative],NEUTRAL
5502,V4A61eTdi_YTBH4761B64g,yb5tB0bsS1sYN9szDbe9Gw,"Shortly after moving to Dormont, while browsin...",2Igiyhi82R8_QaaPplRmCw,2014-09-18,review,0,1,1,1,1,NEUTRAL,[negative],[negative],[negative],[negative]


In [98]:
pd.value_counts(df_.business_id, sort = True)

2SwC8wqpZC4B9iFVTgYT9A    263
IxQ1ATP_Wg_QujO9nywzcQ    149
McikHxxEqZ2X0joaRNKlaw    140
2Igiyhi82R8_QaaPplRmCw    139
PzG0XmAmtXFs6Fdnubf41g    136
93UCt9OgWYmypZCpXtUu8g    130
usSROUMiAgPcxV3Ym0OxMQ    110
83_p9vIqOq1TUIf3_GD7bA    108
qUeBDN0D8SC93KeAHhiL-g    106
wfCLz9-aeShbPB7CaTY-qQ     98
5Udv4CASAXB2eH66TefWPQ     98
sbW8qHJgzEIH42B0S-3New     96
WWlNzfSllSQhr4whj0W_ng     91
q_BKmbdlYfQJroJVHfYMUQ     91
EoAY1JSVeJriBzId8lUQwQ     89
2PfavOTufsPCRdYm-bFcpw     88
JDDeaNfb0JXD1NbznSIC9g     86
P1fJb2WQ1mXoiudj8UE44w     79
lHP5STO7aomlEQixH8m1aA     73
-iVcmhjfq79QoTWzrb6pKg     72
l_Wey2YC6NvVUIH8vTygGg     64
b9WZJp5L1RZr4F1nxclOoQ     64
ZL6DNzyenFm9Kpz6uADlNg     60
Z101WZaISochG6g7tI2ZVw     60
YRDKsBFYP7G0E8BLJKIkvA     56
5GS5sBSscgcex0IFTDL9Bg     56
6ilJq_05xRgek_8qUp36-g     54
Ts4xsKPU7FNPPZRj-nRjIg     52
6p9TlP2I8S4agxYMNFSDjg     50
eznh0XjVtv924K-KYx_ilQ     49
                         ... 
vTHVf88tai9n0902QzZiig      3
1_lU0-eSWJCRvNGk78Zh9Q      3
W7-Ybhe6JW

In [102]:
print pd.value_counts(df_['Clust0_Sent'][df_['business_id'] == "2SwC8wqpZC4B9iFVTgYT9A"])
print pd.value_counts(df_['Clust1_Sent'][df_['business_id'] == "2SwC8wqpZC4B9iFVTgYT9A"])
print pd.value_counts(df_['Clust2_Sent'][df_['business_id'] == "2SwC8wqpZC4B9iFVTgYT9A"])
print pd.value_counts(df_['Clust3_Sent'][df_['business_id'] == "2SwC8wqpZC4B9iFVTgYT9A"])
print pd.value_counts(df_['Clust4_Sent'][df_['business_id'] == "2SwC8wqpZC4B9iFVTgYT9A"])

[negative]    131
NEUTRAL        76
[positive]     56
Name: Clust0_Sent, dtype: int64
[negative]    153
[positive]     85
NEUTRAL        25
Name: Clust1_Sent, dtype: int64
[negative]    120
NEUTRAL        85
[positive]     58
Name: Clust2_Sent, dtype: int64
[negative]    142
[positive]     67
NEUTRAL        54
Name: Clust3_Sent, dtype: int64
[negative]    135
[positive]     66
NEUTRAL        62
Name: Clust4_Sent, dtype: int64


In [106]:
pd.value_counts(df_.Clust_Sent)

[negative]    2732
NEUTRAL       1433
[positive]    1338
Name: Clust4_Sent, dtype: int64

In [108]:
print pd.value_counts(df_['Clust0_Sent'][df_['business_id'] == "IxQ1ATP_Wg_QujO9nywzcQ"])
print pd.value_counts(df_['Clust1_Sent'][df_['business_id'] == "IxQ1ATP_Wg_QujO9nywzcQ"])
print pd.value_counts(df_['Clust2_Sent'][df_['business_id'] == "IxQ1ATP_Wg_QujO9nywzcQ"])
print pd.value_counts(df_['Clust3_Sent'][df_['business_id'] == "IxQ1ATP_Wg_QujO9nywzcQ"])
print pd.value_counts(df_['Clust4_Sent'][df_['business_id'] == "IxQ1ATP_Wg_QujO9nywzcQ"])

[negative]    60
NEUTRAL       58
[positive]    31
Name: Clust0_Sent, dtype: int64
[negative]    92
[positive]    52
NEUTRAL        5
Name: Clust1_Sent, dtype: int64
[negative]    67
NEUTRAL       48
[positive]    34
Name: Clust2_Sent, dtype: int64
NEUTRAL       64
[negative]    61
[positive]    24
Name: Clust3_Sent, dtype: int64
NEUTRAL       63
[negative]    60
[positive]    26
Name: Clust4_Sent, dtype: int64


In [113]:
df_.head()

,user_id,review_id,text,business_id,date,type,Cluster0,Cluster1,Cluster2,Cluster3,Cluster4,Clust0_Sent,Clust1_Sent,Clust2_Sent,Clust3_Sent,Clust4_Sent
0,PUFPaY9KxDAcGqfsorJp3Q,Ya85v4eqdd6k9Od8HbQjyA,"Mr Hoagie is an institution. Walking in, it do...",5UmKMjUEUNdYWqANhGckJw,2012-08-01,review,1,1,0,0,0,[positive],[positive],NEUTRAL,NEUTRAL,NEUTRAL
1,Iu6AxdBYGR4A0wspR9BYHA,KPvLNJ21_4wbYNctrOwWdQ,Excellent food. Superb customer service. I mis...,5UmKMjUEUNdYWqANhGckJw,2014-02-13,review,0,1,0,0,0,NEUTRAL,[positive],NEUTRAL,NEUTRAL,NEUTRAL
2,auESFwWvW42h6alXgFxAXQ,fFSoGV46Yxuwbr3fHNuZig,Yes this place is a little out dated and not o...,5UmKMjUEUNdYWqANhGckJw,2015-10-31,review,1,0,0,0,0,[negative],NEUTRAL,NEUTRAL,NEUTRAL,NEUTRAL
3,qiczib2fO_1VBG8IoCGvVg,pVMIt0a_QsKtuDfWVfSk2A,PROS: Italian hoagie was delicious. Friendly ...,5UmKMjUEUNdYWqANhGckJw,2015-12-26,review,1,1,0,0,0,[negative],[negative],NEUTRAL,NEUTRAL,NEUTRAL
4,qEE5EvV-f-s7yHC0Z4ydJQ,AEyiQ_Y44isJmNbMTyoMKQ,First the only reason this place could possibl...,5UmKMjUEUNdYWqANhGckJw,2016-04-08,review,1,1,0,0,0,[negative],[negative],NEUTRAL,NEUTRAL,NEUTRAL
